In [ ]:
# TimeSeriesMultiReg

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  6 14:44:31 2018
@author: ahellman
"""
import numpy as np
import pandas as pd
import itertools
#import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
#from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from fbprophet import Prophet
#from fbprophet.diagnostics import cross_validation
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA


class TimeSeriesMultiReg():
    
    def __init__(self):
        pass
    
    
    def _Run_Regressions(
        self,
        df,
        forecastDays,
        region="None",
        forecasts={"holtwinters", "arima", "sarima", "prophet"}
    ):
        """This is the main working function to:
            1. break out the arrays from the dataset
            2. Call the other functions
            3. Run the MAPE on each of the functions"""
        
        df = self.format_df(df)
        

        actualsDict = self.dimensionDictionary(df)
        
        actualsDictKey = []
        for key in actualsDict:
            actualsDictKey.append(key)  
        
        # Fill dictionary with unique intersections of the actuals data:
        for i in range(0, len(actualsDictKey)):
            dfFiltered = df.iloc[0:,1].isin(actualsDictKey[i])
            for j in range(1, len(df.columns)-1):
                dfFiltered &= df.iloc[0:,j].isin(actualsDictKey[i])
            
            actualsDict[actualsDictKey[i]] = df[dfFiltered]
        # Fill dictionary with unique uintersections of the actuals data:

        # Dict mapping subdimensions to (model, forecast) tuples.
        # model, forecast = forecast_results[subdimension] <- how to call.
        forecast_results = {}

        for subDimension in actualsDictKey:
            # Initialize the list of forecast results for the subdimension.
            forecast_results[subDimension] = []
            if "prophet" in forecasts:
                try:
                    if region == "None":
                        forecast_results[subDimension].append((
                            "Prophet",
                            self.ProphetFxnNone(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        ))
                    elif region == "NA":
                        forecast_results[subDimension].append((
                            "Prophet",
                            self.ProphetFxnNA(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        ))
                    elif region == "UK":
                        forecast_results[subDimension].append((
                            "Prophet",
                            self.ProphetFxnUK(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        ))
                    elif region == "FR":
                        forecast_results[subDimension].append((
                            "Prophet",
                            self.ProphetFxnUK(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        ))
                    elif region == "CE":
                        forecast_results[subDimension].append((
                            "Prophet",
                            self.ProphetFxnCE(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        ))
                    elif region == "SoEu":
                        forecast_results[subDimension].append((
                            "Prophet",
                            self.ProphetFxnSoEu(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        ))
                except Exception:
                    pass  # If we can't run this without generating exceptions, skip.
            
            if "holtwinters" in forecasts:
                try:
                    forecast_results[subDimension].append(
                        (
                            "HW",
                            self.HWFxn(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        )
                    )
                except Exception:
                    pass  # If we can't run this without generating exceptions, skip.
            
            if "sarima" in forecasts:
                try:
                    forecast_results[subDimension].append(
                        (
                            "Sarima",
                            self.SarimaFxn(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        )
                    )
                except Exception:
                    pass  # If we can't run this without generating exceptions, skip.
            
            if "arima" in forecasts:
                try:
                    forecast_results[subDimension].append(
                        (
                            "Arima",
                            self.ArimaFxn(
                                actualsDict[subDimension],
                                forecastDays
                            )
                        )
                    )
                except Exception:
                    pass  # If we can't run this without generating exceptions, skip.
        
        # Bail if no forecasts were executed.
        if not forecast_results:
            raise ValueError("Unable to execute any forecasts.")
            
        
        # A dictionary mapping the subdimensions to a list of (model, MAPE) tuples.
        # model, mape = mape_results[subdimension] <- how to call.
        mape_results = {}
        for subDimension in actualsDictKey:
            # Initialize to an empty list for the specified subdimension.
            mape_results[subDimension] = []
            for model, forecast in forecast_results[subDimension]:
                mape_results[subDimension].append((
                    model, 
                    self.mapeCalc(
                        forecast[:len(actualsDict[subDimension].groupby('ds', as_index=False).sum())],
                        actualsDict[subDimension].groupby('ds', as_index=False).sum()['y']
                    )
                ))
        
        # A dictionary mapping the subdimension to a tuple of (model, mape, forecast)
        # model, mape, forecast = bestMapes[dimension] <- Usage.
        bestModels = {}
        for subDimension in actualsDictKey:
            bestModel = None
            for model_forecast, model_mape in zip(forecast_results[subDimension], mape_results[subDimension]):
                model, forecast = model_forecast
                model, mape = model_mape

                if (bestModel is None) or ((bestModel[1] < mape).all()):
                    bestModel = (model, mape, forecast)
            bestModels[subDimension] = bestModel

        return bestModels


    def format_df(self, df):
        if 'ds' not in df.columns:
            try:
                df = df.rename(columns={'Date': 'ds'})
            except:
                df = df.rename(columns={'date': 'ds'})
        
        if 'y' not in df.columns:
            try:
                df = df.rename(columns={df.columns[len(df.columns)-1:][0]: 'y'})
            except:
                pass
        
        return df

    
    def mapeCalc(self, pf, xf):

        dff = pd.concat([pf,xf],axis=1)
        dff.columns = ['predict_arima', 'real']
        dff['abspctdiff'] = abs(dff['predict_arima']-dff['real'])/dff['real']
        mape_arima = np.mean(dff['abspctdiff'])
        
        return mape_arima
    

    def dimensionDictionary(self, df):
        
        # Build new arrays to store the individual forecasts - all of this will be stored in a dictionary
        actualsDict = {} #Dictionary to hold all the dataframes of the forecasts
        actualsDictKey = [] #Arry to hold all the dictionary keys
    
        uniqueList = [] #Find and store all the uniqe dimensions of the data
        for i in range(0,len(df.columns)):
            if df.columns[i] != 'ds' and df.columns[i] != 'y':
                uniqueList.append(pd.unique(df.iloc[0:,i].values))
        
        #Combine the arrays in the uniqueList dic into one array of all combinations:
        iterList = list(itertools.product(*uniqueList))
        for i in iterList:
            actualsDictKey.append(i)
        
        #Combine the arrays in the uniqueList dic into one array of all combinations:
        #for i in range(0, len(uniqueList[0])-1):
        #    actualsDictKey.append(uniqueList[0][i])
        
        #Add an empty array into all the intersectons of the unique values
        for i in range(0, len(actualsDictKey)):
            actualsDict[actualsDictKey[i]] = []
            
        return actualsDict
       
        
    def ProphetFxnNA(self, df, daysForecast):
        
        easter = pd.DataFrame({
          'holiday': 'easterSunday',
          'ds': pd.to_datetime(['2010-04-04','2011-04-24','2012-04-08','2013-03-31','2014-04-20',
                                '2015-04-05','2016-03-27','2017-04-16','2018-04-01','2019-04-21',
                                '2020-04-12','2021-04-04','2022-04-17','2023-04-09','2024-03-31']),
          'lower_window': -2,
          'upper_window': 0,
        })
    
        memorial = pd.DataFrame({
          'holiday': 'memorialMonday',
          'ds': pd.to_datetime(['2010-05-31','2011-05-30','2012-05-28','2013-05-27','2014-05-26',
                                '2015-05-25','2016-05-30','2017-05-29','2018-05-28','2019-05-27',
                                '2020-05-25','2021-05-31','2022-05-30','2023-05-29','2024-05-27']),
          'lower_window': -2,
          'upper_window': 0,
        })
    
        laborday = pd.DataFrame({
          'holiday': 'laborMonday',
          'ds': pd.to_datetime(['2010-09-6','2011-09-5','2012-09-3','2013-09-2','2014-09-1',
                                '2015-09-7','2016-09-5','2017-09-4','2018-09-3','2019-09-2',
                                '2020-09-7','2021-09-6','2022-09-5','2023-09-4','2024-09-2']),
          'lower_window': -2,
          'upper_window': 0,
        })
        
        thxgiving = pd.DataFrame({
          'holiday': 'thanksgiving',
          'ds': pd.to_datetime(['2010-11-25','2011-11-24','2012-11-22','2013-11-28','2014-11-27',
                                '2015-11-26','2016-11-24','2017-11-23','2018-11-22','2019-11-28',
                                '2020-11-26','2021-11-25','2022-11-24','2023-11-23','2024-11-28',]),
          'lower_window': 0,
          'upper_window': 1,
        })
    
        holidays = pd.concat((easter, memorial, laborday, thxgiving))
        
        m = Prophet(holidays=holidays, daily_seasonality = False, yearly_seasonality = True, weekly_seasonality = True,
                   seasonality_mode='multiplicative')
        m.fit(df)
        future = m.make_future_dataframe(periods=daysForecast) #Make a new dataframe that predicts the next number of days    
        forecast = m.predict(future)
        return forecast['yhat']

    
    def ProphetFxnCE(self, df, daysForecast):
    
        easter = pd.DataFrame({
              'holiday': 'easterSunday',
              'ds': pd.to_datetime(['2010-04-04','2011-04-24','2012-04-08','2013-03-31','2014-04-20',
                                    '2015-04-05','2016-03-27','2017-04-16','2018-04-01','2019-04-21',
                                    '2020-04-12','2021-04-04','2022-04-17','2023-04-09','2024-03-31']),
              'lower_window': -2,
              'upper_window': 1,
            })
        
        ascensionDay = pd.DataFrame({
              'holiday': 'ascensionDay',
              'ds': pd.to_datetime(['2014-05-29','2015-05-14','2016-05-05','2017-05-25','2018-05-10',
                                    '2019-05-30','2020-05-21','2021-05-13','2022-05-26','2023-05-18','2024-05-09',]),
              'lower_window': 0,
              'upper_window': 2,
            })
        
        whit = pd.DataFrame({
              'holiday': 'whitMonday',
              'ds': pd.to_datetime(['2014-06-09','2015-05-25','2016-05-16','2017-06-05','2018-05-21',
                                    '2019-06-10','2020-06-01','2021-05-24','2022-06-06','2023-05-29','2024-05-20',]),
              'lower_window': -1,
              'upper_window': 0,
            })
        
        holidays = pd.concat((easter, ascensionDay, whit))
    
        m = Prophet(holidays=holidays, daily_seasonality = False, yearly_seasonality = True, weekly_seasonality = True,
                   seasonality_mode='multiplicative')
        m.fit(df)
        future = m.make_future_dataframe(periods=daysForecast) #Make a new dataframe that predicts the next number of days    
        forecast = m.predict(future)
        return forecast['yhat']
    

    def ProphetFxnFR(self, df, daysForecast):
        
        easter = pd.DataFrame({
              'holiday': 'easterSunday',
              'ds': pd.to_datetime(['2010-04-04','2011-04-24','2012-04-08','2013-03-31','2014-04-20',
                                    '2015-04-05','2016-03-27','2017-04-16','2018-04-01','2019-04-21',
                                    '2020-04-12','2021-04-04','2022-04-17','2023-04-09','2024-03-31']),
              'lower_window': -2,
              'upper_window': 1,
            })
        
        ascensionDay = pd.DataFrame({
              'holiday': 'ascensionDay',
              'ds': pd.to_datetime(['2014-05-29','2015-05-14','2016-05-05','2017-05-25','2018-05-10',
                                    '2019-05-30','2020-05-21','2021-05-13','2022-05-26','2023-05-18','2024-05-09',]),
              'lower_window': 0,
              'upper_window': 2,
            })
        
        whit = pd.DataFrame({
              'holiday': 'whitMonday',
              'ds': pd.to_datetime(['2014-06-09','2015-05-25','2016-05-16','2017-06-05','2018-05-21',
                                    '2019-06-10','2020-06-01','2021-05-24','2022-06-06','2023-05-29','2024-05-20',]),
              'lower_window': -1,
              'upper_window': 0,
            })
        
        holidays = pd.concat((easter, ascensionDay, whit))
        
        m = Prophet(holidays=holidays, daily_seasonality = False, yearly_seasonality = True, weekly_seasonality = True,
                   seasonality_mode='multiplicative')
        m.fit(df)
        future = m.make_future_dataframe(periods=daysForecast) #Make a new dataframe that predicts the next number of days    
        forecast = m.predict(future)
        return forecast['yhat']
    

    def ProphetFxnNone(self, df, daysForecast):
    
        m = Prophet(daily_seasonality = False, yearly_seasonality = True, weekly_seasonality = True,
                   seasonality_mode='multiplicative')
        m.fit(df)
        future = m.make_future_dataframe(periods=daysForecast) #Make a new dataframe that predicts the next number of days    
        forecast = m.predict(future)
        return forecast['yhat']
    

    def ProphetFxnSoEu(self, df, daysForecast):
        
        easter = pd.DataFrame({
              'holiday': 'easterSunday',
              'ds': pd.to_datetime(['2010-04-04','2011-04-24','2012-04-08','2013-03-31','2014-04-20',
                                    '2015-04-05','2016-03-27','2017-04-16','2018-04-01','2019-04-21',
                                    '2020-04-12','2021-04-04','2022-04-17','2023-04-09','2024-03-31']),
              'lower_window': -2,
              'upper_window': 1,
            })
        
        holidays = pd.concat((easter))
        
        m = Prophet(holidays=holidays, daily_seasonality = False, yearly_seasonality = True, weekly_seasonality = True,
                   seasonality_mode='multiplicative')
        m.fit(df)
        future = m.make_future_dataframe(periods=daysForecast) #Make a new dataframe that predicts the next number of days    
        forecast = m.predict(future)
        return forecast['yhat']
    
    
    def ProphetFxnUK(self, df, daysForecast):
    
        easter = pd.DataFrame({
              'holiday': 'easterSunday',
              'ds': pd.to_datetime(['2015-04-05','2016-03-27','2017-04-16','2018-04-01','2019-04-21',
                                    '2020-04-12','2021-04-04','2022-04-17','2023-04-09','2024-03-31','2025-04-20']),
            
              'lower_window': -2,
              'upper_window': 1,
            })
        
        #First monday in may
        mayDay = pd.DataFrame({
              'holiday': 'mayday',
              'ds': pd.to_datetime(['2015-05-04','2016-05-02','2017-05-01','2018-05-07','2019-05-06',
                                    '2020-05-04','2021-05-03','2022-05-02','2023-05-01','2024-05-06','2025-05-05']),
              'lower_window': -1,
              'upper_window': 0,
            })
        
        #Last monday in may
        springBank = pd.DataFrame({
              'holiday': 'springbank',
              'ds': pd.to_datetime(['2015-05-25','2016-05-30','2017-05-29','2018-05-28','2019-05-27',
                                    '2020-05-25','2021-05-31','2022-05-30','2023-05-29','2024-05-27','2025-05-26']),
              'lower_window': -1,
              'upper_window': 0,
            })
        
        summerbank = pd.DataFrame({
              'holiday': 'springbank',
              'ds': pd.to_datetime(['2015-08-31','2016-08-29','2017-08-28','2018-08-27','2019-08-26','2020-08-31',
                                    '2021-08-30','2022-08-29','2023-08-28','2024-08-26','2025-08-25']),
              'lower_window': -1,
              'upper_window': 0,
        })
        holidays = pd.concat((easter, mayDay, springBank, summerbank))
        
        m = Prophet(holidays=holidays, daily_seasonality = False, yearly_seasonality = True, weekly_seasonality = True,
                   seasonality_mode='multiplicative')
        m.fit(df)
        future = m.make_future_dataframe(periods=daysForecast) #Make a new dataframe that predicts the next number of days    
        forecast = m.predict(future)
        return forecast['yhat']
    
    
    def SarimaFxn(self, df, fcstdays):

        series = df['y']
    
        # evaluate parameters
        p_values = range(0, 3)
        d_values = range(0, 2)
        q_values = range(0, 3)
        P_values = range(0, 3)
        D_values = range(0, 2)
        Q_values = range(0, 3)
        m = 7 #weekly seasonality
    
        fcstdays = 365
    
        best_score, best_params, best_seasonal_params = float("inf"), None, None
        for p in p_values:
            for d in d_values:
                for q in q_values:
                    for P in P_values:
                        for D in D_values:
                            for Q in Q_values:
                                order = (p,d,q)
                                sorder = (P,D,Q,m)
                                try:
                                    mse = self.evaluate_sarima_model(series, order, sorder, fcstdays)
                                    if mse < best_score:
                                        best_score, best_params, best_seasonal_params = mse, order, sorder
                                except:
                                    # TODO: Logging.
                                    pass
        #print('Best SARIMA %s %s MSE=%.3f' % (best_params, best_seasonal_params, best_score))
        #print('Best Params: ', best_params)
    
    
        # apply model
        model = sm.tsa.statespace.SARIMAX(series,
                                          order=(best_params[0],best_params[1],best_params[2]),
                                          seasonal_order=(best_seasonal_params[0],best_seasonal_params[1],best_seasonal_params[2],m),
                                          enforce_stationarity=True,
                                          enforce_invertibility=True)
        model_fit = model.fit(transparams=True, maxiter=50, method='newton')
        print(model_fit.summary())
    
        pred_fcst = model_fit.predict(start=0, end=len(df['y'])-1, dynamic=False)
        sarima_forecast = model_fit.forecast(fcstdays)
        sarima_series = pred_fcst.append(sarima_forecast)
    
        return sarima_series

    
    def evaluate_sarima_model(self, X, order, sorder, fcstdays):
        # make predictions
        predictions = list()
        model = sm.tsa.statespace.SARIMAX(X,order=order,
                                          seasonal_order=sorder,
                                          enforce_stationarity=True,
                                          enforce_invertibility=True)
        model_fit = model.fit(transparams=True, maxiter=50, method='newton')
        predictions = model_fit.predict(start=0, end=len(X)-1, dynamic=False)
        #yhat = model_fit.forecast(fcstdays)    
        # calculate out of sample error
        error = mean_squared_error(X, predictions)
        print('error = ', error)
        return error
    
    
#    def HoltWinters(self, df, daysForecast):
#
#        seasonalP = 365
#        m_mul = ExponentialSmoothing(df['y'], seasonal='mul', seasonal_periods=seasonalP).fit()
#        hwForecast = m_mul.forecast(daysForecast)
#        HWfull = m_mul.fittedvalues.append(hwForecast)
#
#        return HWfull
    
    def HWFxn(self, df, fcstdays):

        series = df['y']
    
        # evaluate parameters
        t_params = ['add', 'mul', None]
        d_params = [True, False]
        s_params = ['add', 'mul', None]
        p = 365
        b_params = [True, False]
        r_params = [True, False]
           
        #t_params = ['add']
        #d_params = [True]
        #s_params = ['add']
        #p = 365
        #b_params = [True]
        #r_params = [True]

        best_score, best_params = float("inf"), None
        series = series.astype('double')
        for t in t_params:
            for d in d_params:
                for s in s_params:
                    for b in b_params:
                        for r in r_params:
                            params = (t,d,s,p,b,r)
                            try:
                                mse = self.evaluate_hw_model(series, params, fcstdays)
                                if mse < best_score:
                                    best_score, best_params = mse, params
                                #print('HW MSE=%.3f' % (mse))
                                print('HW params', params)
                            except:
                                pass  # TODO: Add logging.
        # apply model
        model = ExponentialSmoothing(series, trend=best_params[0], damped=best_params[1], seasonal=best_params[2], seasonal_periods=best_params[3])
        model_fit = model.fit(optimized=True, use_boxcox=best_params[4], remove_bias=best_params[5])
        print(model_fit.summary())
    
        pred_fcst = model_fit.fittedvalues
        hw_forecast = model_fit.forecast(fcstdays)
        hw_series = pred_fcst.append(hw_forecast)
    
        return hw_series

    
    def evaluate_hw_model(self, X, hw_params, fcstdays):
        (t,d,s,p,b,r) = hw_params
        # make predictions
        predictions = list()
        model = ExponentialSmoothing(X, trend=t, damped=d, seasonal=s, seasonal_periods=365)
        model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
        predictions = model_fit.fittedvalues
        #yhat = model_fit.forecast(fcstdays)
        # calculate out of sample error
        error = mean_squared_error(X, predictions)
        print('error = ', error)
        print('hw_params = ', hw_params)
        return error
    
    def ArimaFxn(self, df, fcstdays):

       series = df['y']

       # evaluate parameters
       p_values = range(0, 6)
       d_values = range(0, 2)
       q_values = range(0, 6)

       fcstdays = 365

       best_score, best_params = float("inf"), None
       for p in p_values:
           for d in d_values:
               for q in q_values:
                    order = (p,d,q)
                    try:
                        mse = self.evaluate_arima_model(series, order, fcstdays)
                        if mse < best_score:
                            best_score, best_params = mse, order
                            print('ARIMA %s %s MSE=%.3f' % (order, mse))
                            print('Best ARIMA %s MSE=%.3f' % (best_params, best_score))
                            print('Best Params: ', best_params)
                    except:
                        pass  # TODO: Add logging.

       # apply model
       model = ARIMA(series, order=(best_params[0],best_params[1],best_params[2]))
       model_fit = model.fit(transparams=True)
       print(model_fit.summary())

       pred_fcst = model_fit.predict(start=1, end=len(df['y'])-1, dynamic=False)
       arima_forecast = model_fit.forecast(fcstdays)[0]
       af = pd.Series(arima_forecast)
       arima_series = pred_fcst.append(af)

       return arima_series


    def evaluate_arima_model(self, X, order, fcstdays):
       # make predictions
       predictions = list()
       model = ARIMA(X,order=order)
       model_fit = model.fit(transparams=True)
       predictions = model_fit.predict(start=1, end=len(X), dynamic=False)
       #yhat = model_fit.forecast(fcstdays)
       # calculate out of sample error
       error = mean_squared_error(X, predictions)
       print('error = ', error)
       return error

In [2]:
import pandas as pd
data = pd.read_csv('NA_NB_1_20181126.csv')
data.shape

(7414, 3)

In [3]:
# from time_series_multireg import TimeSeriesMultiReg
model = TimeSeriesMultiReg()

In [6]:
import warnings
warnings.filterwarnings("ignore")
#x_arima = model._Run_Regressions(data, 365, forecasts={"arima"})
#x_all = model._Run_Regressions(data, 365)
#x_hw_arima_prophet = model._Run_Regressions(data, 365, forecasts={"holtwinters","arima","prophet"})
x_hw = model._Run_Regressions(data, 365, forecasts={"holtwinters"})
#x_sarima = model._Run_Regressions(data, 365, forecasts={"sarima"})
#x_prophet = model._Run_Regressions(data, 365, forecasts={"prophet"})


error =  203595.1973573921
hw_params =  ('add', True, 'add', 365, True, True)
HW params ('add', True, 'add', 365, True, True)
error =  203693.7688550254
hw_params =  ('add', True, 'add', 365, True, False)
HW params ('add', True, 'add', 365, True, False)
error =  187318.74091270214
hw_params =  ('add', True, 'add', 365, False, True)
HW params ('add', True, 'add', 365, False, True)
error =  187369.35468452613
hw_params =  ('add', True, 'add', 365, False, False)
HW params ('add', True, 'add', 365, False, False)
error =  231244.59853741527
hw_params =  ('add', True, 'mul', 365, True, True)
HW params ('add', True, 'mul', 365, True, True)
error =  231871.3793221391
hw_params =  ('add', True, 'mul', 365, True, False)
HW params ('add', True, 'mul', 365, True, False)
error =  341659.0658142225
hw_params =  ('add', True, 'mul', 365, False, True)
HW params ('add', True, 'mul', 365, False, True)
error =  345627.32081249537
hw_params =  ('add', True, 'mul', 365, False, False)
HW params ('add', True

error =  88269.71371133227
hw_params =  ('add', True, 'mul', 365, False, True)
HW params ('add', True, 'mul', 365, False, True)
error =  88312.30993060968
hw_params =  ('add', True, 'mul', 365, False, False)
HW params ('add', True, 'mul', 365, False, False)
error =  71324.91782755563
hw_params =  ('add', True, None, 365, True, True)
HW params ('add', True, None, 365, True, True)
error =  71324.94314769079
hw_params =  ('add', True, None, 365, True, False)
HW params ('add', True, None, 365, True, False)
error =  71220.81724887138
hw_params =  ('add', True, None, 365, False, True)
HW params ('add', True, None, 365, False, True)
error =  71220.82239039056
hw_params =  ('add', True, None, 365, False, False)
HW params ('add', True, None, 365, False, False)
error =  89883.7966650467
hw_params =  ('add', False, 'add', 365, True, True)
HW params ('add', False, 'add', 365, True, True)
error =  89902.71105529903
hw_params =  ('add', False, 'add', 365, True, False)
HW params ('add', False, 'add',

error =  2591.642831729838
hw_params =  ('add', True, None, 365, False, True)
HW params ('add', True, None, 365, False, True)
error =  2591.6606353100065
hw_params =  ('add', True, None, 365, False, False)
HW params ('add', True, None, 365, False, False)
error =  2482.7347996941753
hw_params =  ('add', False, 'add', 365, True, True)
HW params ('add', False, 'add', 365, True, True)
error =  2483.469539912423
hw_params =  ('add', False, 'add', 365, True, False)
HW params ('add', False, 'add', 365, True, False)
error =  2168.11157717159
hw_params =  ('add', False, 'add', 365, False, True)
HW params ('add', False, 'add', 365, False, True)
error =  2168.131211947246
hw_params =  ('add', False, 'add', 365, False, False)
HW params ('add', False, 'add', 365, False, False)
error =  5471.369429002714
hw_params =  ('add', False, 'mul', 365, True, True)
HW params ('add', False, 'mul', 365, True, True)
error =  5495.280343877486
hw_params =  ('add', False, 'mul', 365, True, False)
HW params ('add',

error =  93587.23535069394
hw_params =  ('mul', True, 'add', 365, True, True)
HW params ('mul', True, 'add', 365, True, True)
error =  94520.99673666914
hw_params =  ('mul', True, 'add', 365, True, False)
HW params ('mul', True, 'add', 365, True, False)
error =  43140.922298778205
hw_params =  ('mul', True, 'add', 365, False, True)
HW params ('mul', True, 'add', 365, False, True)
error =  43186.94369554753
hw_params =  ('mul', True, 'add', 365, False, False)
HW params ('mul', True, 'add', 365, False, False)
error =  38567.332339185064
hw_params =  ('mul', True, None, 365, True, True)
HW params ('mul', True, None, 365, True, True)
error =  38569.898638542385
hw_params =  ('mul', True, None, 365, True, False)
HW params ('mul', True, None, 365, True, False)
error =  38574.25734147752
hw_params =  ('mul', True, None, 365, False, True)
HW params ('mul', True, None, 365, False, True)
error =  38578.21963749632
hw_params =  ('mul', True, None, 365, False, False)
HW params ('mul', True, None, 

error =  12967.791367242979
hw_params =  ('mul', False, 'add', 365, True, True)
HW params ('mul', False, 'add', 365, True, True)
error =  12998.41095277697
hw_params =  ('mul', False, 'add', 365, True, False)
HW params ('mul', False, 'add', 365, True, False)
error =  4891.850181252633
hw_params =  ('mul', False, 'mul', 365, True, True)
HW params ('mul', False, 'mul', 365, True, True)
error =  4893.368762750975
hw_params =  ('mul', False, 'mul', 365, True, False)
HW params ('mul', False, 'mul', 365, True, False)
error =  13614.892606731792
hw_params =  ('mul', False, 'mul', 365, False, True)
HW params ('mul', False, 'mul', 365, False, True)
error =  13845.922792733822
hw_params =  ('mul', False, 'mul', 365, False, False)
HW params ('mul', False, 'mul', 365, False, False)
error =  8245.75295735526
hw_params =  ('mul', False, None, 365, True, True)
HW params ('mul', False, None, 365, True, True)
error =  8251.109395682828
hw_params =  ('mul', False, None, 365, True, False)
HW params ('mul

error =  87096.85897632471
hw_params =  ('mul', False, None, 365, True, True)
HW params ('mul', False, None, 365, True, True)
error =  87101.42219549438
hw_params =  ('mul', False, None, 365, True, False)
HW params ('mul', False, None, 365, True, False)
error =  86735.60112961473
hw_params =  ('mul', False, None, 365, False, True)
HW params ('mul', False, None, 365, False, True)
error =  86943.12148673958
hw_params =  ('mul', False, None, 365, False, False)
HW params ('mul', False, None, 365, False, False)
error =  118144.44471517143
hw_params =  (None, False, 'add', 365, True, True)
HW params (None, False, 'add', 365, True, True)
error =  118198.33466923171
hw_params =  (None, False, 'add', 365, True, False)
HW params (None, False, 'add', 365, True, False)
error =  82450.78626860197
hw_params =  (None, False, 'add', 365, False, True)
HW params (None, False, 'add', 365, False, True)
error =  82478.25141542999
hw_params =  (None, False, 'add', 365, False, False)
HW params (None, False, 

error =  95335.29830112305
hw_params =  (None, False, 'add', 365, True, True)
HW params (None, False, 'add', 365, True, True)
error =  95342.5792704042
hw_params =  (None, False, 'add', 365, True, False)
HW params (None, False, 'add', 365, True, False)
error =  92896.54306673331
hw_params =  (None, False, 'add', 365, False, True)
HW params (None, False, 'add', 365, False, True)
error =  92901.62500514052
hw_params =  (None, False, 'add', 365, False, False)
HW params (None, False, 'add', 365, False, False)
error =  100963.14084998684
hw_params =  (None, False, 'mul', 365, True, True)
HW params (None, False, 'mul', 365, True, True)
error =  101096.77593720543
hw_params =  (None, False, 'mul', 365, True, False)
HW params (None, False, 'mul', 365, True, False)
error =  102397.9759470665
hw_params =  (None, False, 'mul', 365, False, True)
HW params (None, False, 'mul', 365, False, True)
error =  102742.01545308845
hw_params =  (None, False, 'mul', 365, False, False)
HW params (None, False, 

In [2]:
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-64-e2abd6c24e24> in <module>()
      2 warnings.filterwarnings("ignore")
      3 #x_arima = model._Run_Regressions(data, 365, forecasts={"arima"})
----> 4 x_all = model._Run_Regressions(data, 365)

<ipython-input-60-85726345dc46> in _Run_Regressions(self, df, forecastDays, region, forecasts)
    184                 model, mape = model_mape
    185 
--> 186                 if (bestModel is None) or ((bestModel[1] < mape).all()):
    187                     bestModel = (model, mape, forecast)
    188             bestModels[subDimension] = bestModel

AttributeError: 'bool' object has no attribute 'all'
    
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-65-1d6684c7257f> in <module>()
      3 #x_arima = model._Run_Regressions(data, 365, forecasts={"arima"})
      4 #x_all = model._Run_Regressions(data, 365)
----> 5 x_hw_arima_prophet = model._Run_Regressions(data, 365, forecasts={"holtwinters", "arima", "prophet"})

<ipython-input-60-85726345dc46> in _Run_Regressions(self, df, forecastDays, region, forecasts)
    184                 model, mape = model_mape
    185 
--> 186                 if (bestModel is None) or ((bestModel[1] < mape).all()):
    187                     bestModel = (model, mape, forecast)
    188             bestModels[subDimension] = bestModel

AttributeError: 'bool' object has no attribute 'all'



In [68]:
x_hw

{('Undefined',): ('HW', 0.42797985288540363, 0      1202.714761
  10     1224.839341
  20     1106.829419
  30     1171.257707
  40     1138.548503
  50     1203.069846
  60     1294.933963
  70     2061.375680
  80     1420.165528
  90     1367.114489
  100    1364.043389
  110    1385.250749
  120    1393.763467
  130    1467.999357
  140    1381.839951
  150    1200.939782
  160    1259.194835
  170    1370.732963
  180    1450.827681
  190    1450.547146
  200    1627.002621
  210    1311.806158
  220    1292.215946
  230    1268.699843
  240    1308.771147
  250    1421.896688
  260    1445.886388
  270    1482.907127
  280    1296.439311
  290    1224.147622
            ...     
  519     336.572606
  520     335.803416
  521     335.036568
  522     334.272051
  523     333.509859
  524     332.749982
  525     331.992411
  526     331.237140
  527     330.484158
  528     329.733458
  529     328.985032
  530     328.238870
  531     327.494966
  532     326.753310
  533     32

In [69]:
x_arima

{('Undefined',): ('Arima', 0.36358295101970234, 10     1172.335955
  20     1073.114499
  30     1149.484953
  40     1108.593687
  50     1168.727902
  60     1230.666145
  70     1910.182783
  80     1184.964141
  90     1242.692988
  100    1251.111778
  110    1269.152043
  120    1271.557411
  130    1331.090285
  140    1245.098356
  150    1118.816503
  160    1201.200379
  170    1281.178886
  180    1322.070153
  190    1304.029888
  200    1451.960059
  210    1163.917165
  220    1204.207090
  230    1189.173536
  240    1227.058092
  250    1312.448678
  260    1306.435256
  270    1331.691627
  280    1175.342666
  290    1152.491664
  300    1171.133271
            ...     
  335    1092.896629
  336    1092.896629
  337    1092.896629
  338    1092.896629
  339    1092.896629
  340    1092.896629
  341    1092.896629
  342    1092.896629
  343    1092.896629
  344    1092.896629
  345    1092.896629
  346    1092.896629
  347    1092.896629
  348    1092.896629
  349    

In [71]:
x_sarima

{('Undefined',): ('Sarima', 0.4395981075052672, 0         0.000000
  10     1173.644595
  20     1086.236987
  30     1150.099747
  40     1123.011468
  50     1177.022268
  60     1247.753423
  70     1915.735129
  80     1385.780862
  90     1343.640156
  100    1338.174204
  110    1361.374529
  120    1371.963986
  130    1475.530133
  140    1327.153314
  150    1239.485466
  160    1272.899783
  170    1362.206091
  180    1405.650009
  190    1401.617311
  200    1446.027011
  210    1352.535640
  220    1304.075279
  230    1293.079406
  240    1315.225839
  250    1395.518663
  260    1412.750504
  270    1454.662720
  280    1302.623529
  290    1246.163600
            ...     
  519     868.592771
  520     884.119258
  521     863.475104
  522     872.653835
  523     882.207215
  524     832.785137
  525     857.981689
  526     867.638516
  527     856.945388
  528     876.166934
  529     877.423587
  530     859.017681
  531     897.412803
  532     846.824600
  533    

In [73]:
x_prophet

{('Undefined',): ('Prophet', 0.46146164262988465, 0      1042.279190
  1      1161.714479
  2      1286.433188
  3      1430.488966
  4      1498.603819
  5      1541.265207
  6      1526.258118
  7      1297.130808
  8      1330.118023
  9      1379.844741
  10     1461.173106
  11     1479.174081
  12     1484.243106
  13     1443.663505
  14     1200.114890
  15     1228.648626
  16     1282.451403
  17     1374.757453
  18     1408.888652
  19     1433.439537
  20     1413.908712
  21     1191.271766
  22     1239.015786
  23     1308.954800
  24     1413.185187
  25     1454.159179
  26     1479.892669
  27     1455.608352
  28     1222.309251
  29     1253.802998
            ...     
  519     535.049605
  520     590.394450
  521     679.747103
  522     705.776858
  523     716.669180
  524     677.766657
  525     430.146036
  526     447.651468
  527     482.703384
  528     548.242204
  529     547.785219
  530     530.613125
  531     463.387549
  532     188.695005
  533  

In [7]:
with open('x_prophet_mult_all.txt', 'w') as f:
    print(x_prophet, file=f)
with open('x_hw_all.txt', 'w') as f:
    print(x_hw, file=f)
with open('x_arima_wider_all.txt', 'w') as f:
    print(x_arima, file=f)
#with open('x_sarima50_nm.txt', 'w') as f:
    #print(x_sarima, file=f)

In [99]:
with open('x_sarima50_nm.txt', 'w') as f:
    print(x_sarima, file=f)

In [104]:
with open('x_sarima50_ncg.txt', 'w') as f:
    print(x_sarima, file=f)

In [ ]:
with open('x_sarima50_newton.txt', 'w') as f:
    print(x_sarima, file=f)

In [ ]:
model.mapeCalc